In [64]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os

path = os.path.join(os.getcwd(), "data")
files = [file for file in os.listdir(path) if file.endswith("csv")]

data = pd.DataFrame()
for file in files:
    file_data = pd.read_csv(os.path.join(path,file))
    file_data["Şirket"] = file[: file.find(" ")]
    data = data.append(file_data)

#Interst Rates
interest_rates = pd.DataFrame([["13 09 2019",	16.50],
                                ["25 10 2019",	14.00],
                                ["13 12 2019",	12.00],
                                ["17 01 2020",	11.25],
                                ["20 02 2020",	10.75],
                                ["18 03 2020",	9.75],
                                ["23 04 2020",	8.75],
                                ["22 05 2020",	8.25],
                                ["25 09 2020",	10.25],
                                ["20 11 2020",	15.00]],
                            columns=["date", "rate"])
interest_rates["date"] = pd.to_datetime(interest_rates["date"], dayfirst=True) 
interest_rates = interest_rates.set_index("date")
interest_rates = (interest_rates / 365) * 7 #convert to weekly interest
interest_rates = interest_rates.sort_index(ascending=False)

# Process Columns
data["Tarih"] = pd.to_datetime(data["Tarih"], dayfirst=True)
data[["Şimdi", "Açılış", "Yüksek", "Düşük"]] = data[
    ["Şimdi", "Açılış", "Yüksek", "Düşük"]
].applymap(lambda x: round(float(str(x).replace(",", ".")), 3))
data["Fark %"] = data["Fark %"].apply(
    lambda x: round(float(str(x).replace(",", ".").replace("%", "")) / 100, 3)
)

data = data[["Şirket", "Tarih", "Şimdi", "Açılış", "Yüksek", "Düşük", "Fark %"]]

# Calculate net return by substracting the weekly inflation rate from weekly return
def get_interst(date_):
    return interest_rates[interest_rates.index <=date_].head(1)["rate"].values[0]
data["net_return"] = data.apply(lambda row: row["Fark %"] - get_interst(row["Tarih"]) ,axis=1)

data = data.sort_values(["Şirket", "Tarih"])

#Mean & Variance for weekly return
data_agg = data.groupby("Şirket").agg({"Fark %": ["mean", "var"]})
data_agg.columns = data_agg.columns.get_level_values(1)
mean_returns = data_agg["mean"]

#Mean & Variance for weekly net return
data_agg_net = data.groupby("Şirket").agg({"net_return": ["mean", "var"]})
data_agg_net.columns = data_agg_net.columns.get_level_values(1)
mean_net_returns = data_agg_net["mean"]

# Cov for weekly return
cov_data = pd.DataFrame()
for company in data["Şirket"].unique():
    current = data[data["Şirket"] == company]["Fark %"].rename(company)
    cov_data = pd.concat([cov_data, current], axis=1)
cov = cov_data.cov()

# Cov for weekly net return
cov_data_net = pd.DataFrame()
for company in data["Şirket"].unique():
    current = data[data["Şirket"] == company]["net_return"].rename(company)
    cov_data_net = pd.concat([cov_data_net, current], axis=1)
cov_net = cov_data_net.cov()

"""
writer = pd.ExcelWriter("risk.xlsx", engine="xlsxwriter")
data.to_excel(writer, sheet_name="Data")
data_agg.to_excel(writer, sheet_name="Data Agg")
cov_data.to_excel(writer, sheet_name="Cov Data")
cov.to_excel(writer, sheet_name="Covariance")

writer.save()
"""
print("Success")


Success


In [48]:
def calculate_return(column):
    return np.prod(column.apply(lambda x: x + 1)) - 1

weekly_returns = cov_data.reset_index(drop=True)
rate_of_total_returns = weekly_returns.apply(calculate_return)
rate_of_total_returns

AEFES   -0.020448
AKBNK   -0.214452
DEVA     1.795690
DGKLB    0.531219
ECILC    0.916592
ENJSA    0.447360
ENKAI    0.186614
GOLTS    0.913725
IHLGM    0.459129
SISE     0.440644
THYAO   -0.103348
TTKOM    0.105694
TTRAK    1.984149
ULKER    0.010131
VESTL    0.711300
dtype: float64

In [49]:
data_agg["mean_for_p1"] = data_agg["mean"].apply(lambda x: max([x, 0]))
data_agg["var_for_p2"] =  data_agg["var"].max() + data_agg["var"].min() - data_agg["var"]

In [50]:
data_agg['Portfolio1'] = data_agg["mean_for_p1"] / data_agg["mean_for_p1"].sum()
data_agg['Portfolio2'] = data_agg["var_for_p2"] / data_agg["var_for_p2"].sum()

p1_weights = data_agg['Portfolio1']
p2_weights = data_agg['Portfolio2']

print("Check weights", p1_weights.sum(), p2_weights.sum())

Check weights 1.0000000000000002 1.0


In [51]:
data_agg.sort_values("var")

,mean,var,mean_for_p1,var_for_p2,Portfolio1,Portfolio2
Şirket,,,,,,
ENKAI,0.004135,0.001673,0.004135,0.009104,0.028667,0.105195
ENJSA,0.008019,0.001804,0.008019,0.008973,0.055600,0.103682
SISE,0.008385,0.002660,0.008385,0.008118,0.058133,0.093794
ULKER,0.001673,0.002933,0.001673,0.007845,0.011600,0.090638
AEFES,0.001135,0.002998,0.001135,0.007780,0.007867,0.089890
TTKOM,0.003731,0.003567,0.003731,0.007211,0.025867,0.083318
AKBNK,-0.002769,0.003918,0.000000,0.006860,0.000000,0.079264
THYAO,0.000115,0.004382,0.000115,0.006396,0.000800,0.073902
VESTL,0.012788,0.004671,0.012788,0.006107,0.088667,0.070561


In [52]:
# multiply weights
def mul_weight(row_column):
    
    return np.multiply(row_column, p1_weights)

cov_p1 = cov.apply(mul_weight, axis=1).apply(mul_weight, axis=0)

In [53]:
p1_parameters = pd.DataFrame.from_dict({"Rate of Return of the portfolio (52w)": [np.matmul(rate_of_total_returns, p1_weights)],
                                        "Mean return of the portfolio (w)": [np.matmul(p1_weights, mean_returns)],
                                        "Variance of the portfolio": [cov_p1.sum().sum()]}).rename({0:"Portfolio 1"})
p1_parameters

,Rate of Return of the portfolio (52w),Mean return of the portfolio (w),Variance of the portfolio
Portfolio 1,1.033112,0.015832,0.002735


In [54]:
# multiply weights
def mul_weight2(row_column):
    
    return np.multiply(row_column, p2_weights)

cov_p2 = cov.apply(mul_weight2, axis=1).apply(mul_weight2, axis=0)

In [55]:

p2_parameters = pd.DataFrame.from_dict({"Rate of Return of the portfolio (52w)": [np.matmul(rate_of_total_returns, p2_weights)],
                                        "Mean return of the portfolio (w)": [np.matmul(p2_weights, mean_returns)],
                                        "Variance of the portfolio": [cov_p2.sum().sum()]}).rename({0:"Portfolio 2"})
p2_parameters

,Rate of Return of the portfolio (52w),Mean return of the portfolio (w),Variance of the portfolio
Portfolio 2,0.40344,0.007155,0.001751


In [56]:
portfolio_parameters = p1_parameters.append(p2_parameters)

# 6.1

In [57]:
portfolio_parameters

,Rate of Return of the portfolio (52w),Mean return of the portfolio (w),Variance of the portfolio
Portfolio 1,1.033112,0.015832,0.002735
Portfolio 2,0.403440,0.007155,0.001751


In [58]:
portfolio_parameters["z-value"] = ((0 - portfolio_parameters["Mean return of the portfolio (w)"]) / 
portfolio_parameters[ "Variance of the portfolio"] )
portfolio_parameters["p-value"] = portfolio_parameters["z-value"].apply(lambda z: stats.norm.sf(abs(z)))


In [59]:
portfolio_parameters

,Rate of Return of the portfolio (52w),Mean return of the portfolio (w),Variance of the portfolio,z-value,p-value
Portfolio 1,1.033112,0.015832,0.002735,-5.788626,3.548233e-09
Portfolio 2,0.403440,0.007155,0.001751,-4.087349,2.181654e-05


In [60]:

interest_rates

,rate
date,
2020-11-20,0.287671
2020-09-25,0.196575
2020-05-22,0.158219
2020-04-23,0.167808
2020-03-18,0.186986
2020-02-20,0.206164
2020-01-17,0.215753
2019-12-13,0.230137
2019-10-25,0.268493


# 6.2